In [3]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix
import glob

In [4]:
path_ds='/net/bmc-lab6/data/lab/kellis/users/khrovati/data/datasets/d10_1038_s41598-023-28429-y/'

In [5]:
path_train='/net/bmc-lab6/data/lab/kellis/users/khrovati/data/cross_system_integration/retina_embryo_organoid_2023/'

## Make AnnData

Load data

In [11]:
# Get all samples, file names based on them
samples=[]
for f in sorted(glob.glob(path_ds+'Seurat_velocyto_monocle*matrix.mtx')):
    samples.append(f.split('Seurat_velocyto_monocle :')[1].replace(':matrix.mtx',''))
print(samples)

['IPS1 ', 'IPS2', 'IPS3', 'IPS4', 'RE1', 'RE2', 'RE3', 'RE4']


In [12]:
# Mapping between file name samples and annotation samples
sample_map={
    'IPS1 ':'DD13',
    'IPS2':'DD21',
    'IPS3':'DD25',
    'IPS4':'DD29',
    'RE1':'E13',
    'RE2':'P0',
    'RE3':'P5',
    'RE4':'P9',
}

In [ ]:
# Load data
adata={}
for fn,sample in sample_map.items():
    #print(sample)
    var=pd.read_table(path_ds+'Seurat_velocyto_monocle :'+fn+':genes.tsv',header=None,index_col=0)
    var.columns=['gene_symbol']
    obs=pd.read_table(path_ds+'Seurat_velocyto_monocle :'+fn+':barcodes.tsv',header=None,index_col=0)
    obs.index.name=None
    var.index.name=None
    fn_mtx=path_ds+'Seurat_velocyto_monocle :'+fn+':matrix.mtx'
    try:
        mtx=csr_matrix(mmread(fn_mtx)).T
    except ValueError: 
        fn_mtx=path_ds+'Seurat_velocyto_monocle :'+fn+':matrix.mtx'
        mtx=pd.read_table(fn_mtx,header=None,sep=' ').values
        mtx=csr_matrix((mtx[:,2],(mtx[:,1]-1,mtx[:,0]-1)),shape=(obs.shape[0],var.shape[0]))
    adata[sample]=sc.AnnData(mtx,obs=obs,var=var)
adata=sc.concat(adata.values(),label='sample',keys=adata.keys(),index_unique ='-',merge ='same')

In [62]:
adata

AnnData object with n_obs × n_vars = 40299 × 27998
    obs: 'sample'
    var: 'gene_symbol'

Add annotation

In [15]:
anno=pd.read_table(path_ds+'Annotations_cells.csv',sep=',',index_col=0)

In [16]:
anno.index.map(lambda x: x.split('_')[:-1]).value_counts()

[P5, 1]      3733
[P5, 2]      3671
[P9]         3576
[DD25, 2]    3153
[DD25, 1]    2463
[DD29]       2204
[DD13, 2]    1568
[P0, 2]      1517
[E13, 1]     1372
[DD21, 2]    1326
[E13, 2]     1323
[P0, 1]      1227
[DD13, 1]    1129
[DD21, 1]    1105
Name: Cells, dtype: int64

In [17]:
adata.obs.index.map(lambda x: x.split('-')[1:]).value_counts()

[2, P5]      6212
[1, P5]      4513
[1, P9]      4468
[1, DD29]    3993
[2, DD25]    3890
[1, DD25]    3543
[2, DD13]    2291
[2, DD21]    1895
[2, P0]      1761
[1, E13]     1641
[2, E13]     1571
[1, DD13]    1567
[1, DD21]    1498
[1, P0]      1456
dtype: int64

In [18]:
anno.index=anno.index.map(lambda x: '-'.join(
    [x.split('_')[-1],x.split('_')[1] if len(x.split('_'))==3 else '1',x.split('_')[0]]))

In [19]:
anno.index.map(lambda x: x.split('-')[1:]).value_counts()

[1, P5]      3733
[2, P5]      3671
[1, P9]      3576
[2, DD25]    3153
[1, DD25]    2463
[1, DD29]    2204
[2, DD13]    1568
[2, P0]      1517
[1, E13]     1372
[2, DD21]    1326
[2, E13]     1323
[1, P0]      1227
[1, DD13]    1129
[1, DD21]    1105
Name: Cells, dtype: int64

In [97]:
# Find cells that are probably not correct from anno
adata_cells=set(adata.obs_names)
anno_cells=set(anno.index)
print(anno_cells-adata_cells)

{'ACACCCTCAGTTTACG-2-DD21'}


In [99]:
cols=['TimePoint','Model','Annotations']
cells=list(anno_cells&adata_cells)
adata.obs.loc[cells,cols]=anno.loc[cells,cols]

In [25]:
# Fix condition and sample (bio replicates)
adata.obs['sample']=adata.obs.index.map(
    lambda x: x.split('-')[2] +'_'+x.split('-')[1] if len(x.split('-'))==3 else x.split('-')[1]+'_1')

In [28]:
adata.obs['sample'].value_counts(sort=False)

DD13_1    1567
DD13_2    2291
DD21_1    1498
DD21_2    1895
DD25_1    3543
DD25_2    3890
DD29_1    3993
E13_1     1641
E13_2     1571
P0_1      1456
P0_2      1761
P5_1      4513
P5_2      6212
P9_1      4468
Name: sample, dtype: int64

In [32]:
adata.obs['batch']=adata.obs['sample'].map(lambda x: x.split('_')[0])

There are two bio samples per condition, except for the 4th stage in both models. But they were sequenced together:

We generated two biological replicate samples (NaR and RO) for stages I to III and one biological replicate for stage IV. We loaded ~ 15,700 cells for biological replicate 1 (stage I–IV) and ~ 10,000 cells for biological replicate 2 (stage I–III) in a Chromium Controller instrument (10× Genomics, Pleasanton, CA).

In [64]:
sorted(adata.obs['TimePoint'].dropna().unique())

['DD13', 'DD21', 'DD25', 'DD29', 'E13', 'P0', 'P5', 'P9']

In [65]:
adata.obs['stage']=adata.obs['TimePoint'].map({
    'DD13':1, 
    'DD21':2, 
    'DD25':3, 
    'DD29':4, 
    'E13':1, 
    'P0':2, 
    'P5':3, 
    'P9':4
})

In [66]:
adata

AnnData object with n_obs × n_vars = 40299 × 27998
    obs: 'sample', 'TimePoint', 'Model', 'Annotations', 'batch', 'stage'
    var: 'gene_symbol'

Save data

In [70]:
adata.write(path_ds+'adata_Seurat_velocyto_monocle.h5ad')

## Training data

In [6]:
#adata=sc.read(path_ds+'adata_Seurat_velocyto_monocle.h5ad')

In [31]:
adata

AnnData object with n_obs × n_vars = 40299 × 27998
    obs: 'sample', 'TimePoint', 'Model', 'Annotations'
    var: 'gene_symbol'

In [75]:
# Subset to annotated cells
adata_sub=adata[~adata.obs.Annotations.isna(),:].copy()

In [80]:
# Keep not too lowly expressed genes as intersection of the two systems
adata_sub=adata_sub[:,
                    np.array((adata_sub[adata_sub.obs.Model=="iPS",:].X>0).sum(axis=0)>20).ravel()&\
                    np.array((adata_sub[adata_sub.obs.Model=="Retinal",:].X>0).sum(axis=0)>20).ravel()
                   ]

In [81]:
adata_sub.shape

(29366, 14833)

In [82]:
# Normalize and log scale
# Can normalize together as just CPM
sc.pp.normalize_total(adata_sub, target_sum=1e4)
sc.pp.log1p(adata_sub)

/net/bmc-lab5/data/kellis/users/khrovati/miniconda/envs/analysis/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [83]:
hvgs=set(sc.pp.highly_variable_genes(
    adata_sub[adata_sub.obs.Model=="iPS",:], 
    n_top_genes=3000, flavor='cell_ranger', inplace=False, batch_key='sample').query('highly_variable==True').index)&\
set(sc.pp.highly_variable_genes(
    adata_sub[adata_sub.obs.Model=="Retinal",:], 
    n_top_genes=3000, flavor='cell_ranger', inplace=False, batch_key='sample').query('highly_variable==True').index)
print(len(hvgs))

2143


In [84]:
adata_sub=adata_sub[:,list(hvgs)]

In [85]:
del adata_sub.uns

In [86]:
adata_sub.obs['system']=adata_sub.obs['Model'].map({"iPS":0,'Retinal':1})

In [8]:
adata_sub.layers['counts']=adata[adata_sub.obs_names,adata_sub.var_names].X.copy()

In [9]:
adata_sub

AnnData object with n_obs × n_vars = 29366 × 2143
    obs: 'sample', 'TimePoint', 'Model', 'Annotations', 'batch', 'stage', 'system'
    var: 'gene_symbol'
    layers: 'counts'

In [10]:
adata_sub.write(path_train+'combined_HVG.h5ad')

In [7]:
#adata_sub=sc.read(path_train+'combined_HVG.h5ad')